In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(device)

cuda


In [ ]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  
    transforms.Normalize([0.5,0.5,0.5],
                        [0.5,0.5,0.5])
])

In [ ]:
train_path='C:/Users/acer/Desktop/trina/train'
test_path='C:/Users/acer/Desktop/trina/test'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=10, 
    shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=10, 
    shuffle=True
)

In [ ]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:
print(classes)

['banana', 'bucket', 'elephant', 'keyboard', 'orange', 'penguin', 'pizza', 'schoolbus', 'tea', 'tower']


In [ ]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=10):
        super(ConvNet,self).__init__()
        
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU()
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        self.relu2=nn.ReLU()
 
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            
        output=output.view(-1,32*75*75)
        
            
            
        output=self.fc(output)
            
        return output

In [ ]:
model=ConvNet(num_classes=10).to(device)

In [ ]:
optimizer=Adam(model.parameters(),lr=0.0001,weight_decay=0.00001)
loss_function=nn.CrossEntropyLoss()

In [ ]:
num_epochs=5

In [ ]:
best_accuracy=0.0

for epoch in range(num_epochs):
    
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/500
    train_loss=train_loss/500
    
    
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/50

    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint2.model')
        best_accuracy=test_accuracy
    

Epoch: 0 Train Loss: tensor(3.4833) Train Accuracy: 0.378 Test Accuracy: 0.06
Epoch: 1 Train Loss: tensor(1.3994) Train Accuracy: 0.662 Test Accuracy: 0.08
Epoch: 2 Train Loss: tensor(0.6299) Train Accuracy: 0.822 Test Accuracy: 0.02
Epoch: 3 Train Loss: tensor(0.4044) Train Accuracy: 0.868 Test Accuracy: 0.02
Epoch: 4 Train Loss: tensor(0.3245) Train Accuracy: 0.912 Test Accuracy: 0.04
